In [1]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install unstructured
# !pip3 install sentence-transformers

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import JinaEmbeddings


import os

In [2]:
directories = ["Web Scholar PDFs", "Data/About Scottie", "Data/Buggy News", "academic_calendars", "Data/history_of_cmu", "Data/history_of_scs", "Data/Kiltie Band", "Data/lti_faculty", "Data/lti_programs", "program_handbooks", "Data/Tartan Facts"]


In [ ]:
docs = []
for i in directories:
    loader = DirectoryLoader(i, glob="*", exclude="annotation.txt", show_progress=True)
    docs.append(loader.load())

In [ ]:
all_splits = []
for i in docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits.append(text_splitter.split_documents(i))

In [3]:
import pickle as pkl

# with open('noWebScholarSplitDocuments.pkl', "wb") as f:
#     pkl.dump(all_splits[1:], f)


with open('splitDocuments.pkl','rb') as f: 
  all_splits = pkl.load(f)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


# model_name = "Salesforce/SFR-Embedding-Mistral"
# model_kwargs = {"device": "cuda"}
# embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# import chromadb.utils.embedding_functions as embedding_functions
# huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
#     model_name="BAAI/llm-embedder"
# )

model_name = "BAAI/llm-embedder"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)



for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=embeddings, persist_directory="llm-embedder/")